In [12]:
import pandas as pd

In [13]:
df_3ap = pd.read_csv("./df_3ap_step1.csv", header=[0, 1], index_col=None)
len(df_3ap)

691

In [14]:
len(df_3ap.columns)

382

In [15]:
# (*rssi, !org)
# (!*rssi, _)

# 提取与 "*rssi, !org" 相关的列 (一级索引包含 'rssi' 且二级索引不等于 'org')
rssi_not_org_cols = [
    col
    for col in df_3ap.columns
    if "rssi" in col[0] and col[1] != "org" and col[1] != "_"
]

# 提取与 "!*rssi, _" 相关的列 (一级索引不包含 'rssi' 且二级索引等于 '_')
not_rssi_underscore_cols = [
    col for col in df_3ap.columns if "rssi" not in col[0] and col[1] == "_"
]

# 创建新的 DataFrame 包含提取出的列
df_rssi_not_org = df_3ap[rssi_not_org_cols]
df_not_rssi_underscore = df_3ap[not_rssi_underscore_cols]

In [16]:
cols_to_use = [
    # ("test_id", "_"),
    # ("test_dur", "_"),
    # ("loc_id", "_"),
    ("protocol", "_"),
    # ("pkt_len", "_"),
    ("bss_id", "_"),
    # ("ap_name", "_"),
    # ("ap_mac", "_"),
    # ("ap_id", "_"),
    ("pd", "_"),
    ("ed", "_"),
    ("nav", "_"),
    ("eirp", "_"),
    # ("sta_mac", "_"),
    # ("sta_id", "_"),
    # ("seq_time", "_"),
    ("mcs_nss", "_"),
    ("throughput", "_"),
]

catagory_cols = [
    # ("loc_id", "_"),
    ("protocol", "_"),
    ("bss_id", "_"),
    # ("ap_mac", "_"),
    # ("sta_mac", "_"),
    ("mcs_nss", "_"),
]

In [17]:
# count unique values in ("mcs_nss", "_"),
df_3ap[("mcs_nss", "_")].value_counts(), len(df_3ap[("mcs_nss", "_")].unique())

((mcs_nss, _)
 11_2    256
 8_2      79
 4_2      78
 5_2      69
 7_2      63
 9_2      57
 10_2     35
 6_2      34
 5_1      10
 3_2       6
 9_1       1
 0_0       1
 2_2       1
 4_1       1
 Name: count, dtype: int64,
 14)

In [18]:
df_3ap = pd.concat([df_not_rssi_underscore[cols_to_use], df_rssi_not_org], axis=1)

# # deal with catagory columns
# for col in catagory_cols:
#     df_2ap[col] = df_2ap[col].astype("category")

In [19]:
df_dummies = pd.get_dummies(df_3ap[catagory_cols])
new_cols = [(col, "_") for col in df_dummies.columns]
# new col names (org_name, _)
df_dummies.columns = pd.MultiIndex.from_tuples(new_cols)

In [20]:
df_3ap = pd.concat([df_3ap, df_dummies], axis=1)
df_3ap.drop(columns=catagory_cols, inplace=True)

In [21]:
len(df_3ap.columns)

353

In [22]:
# save
df_3ap.to_csv("./df_3ap_final.csv", index=False)